# **Linear regression without using PyTorch built-ins**

### Linear regression model from scratch

In [1]:
import numpy as np
import torch

In [2]:
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')

In [3]:
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
weight = torch.randn(2, 3, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

In [6]:
inputs @ weight.t() + bias

tensor([[ 36.2536, -29.4826],
        [ 42.8841, -47.9727],
        [ 23.9670,   7.2395],
        [ 70.8914, -55.4939],
        [ 20.0058, -40.3066]], grad_fn=<AddBackward0>)

In [7]:
def model(inputs):
    return inputs @ weight.t() + bias

In [8]:
prediction = model(inputs)
print(prediction)

tensor([[ 36.2536, -29.4826],
        [ 42.8841, -47.9727],
        [ 23.9670,   7.2395],
        [ 70.8914, -55.4939],
        [ 20.0058, -40.3066]], grad_fn=<AddBackward0>)


### Loss function

In [9]:
diff0 = prediction - targets
torch.sum(diff0 * diff0) / diff0.numel()

tensor(8312.6162, grad_fn=<DivBackward0>)

In [10]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
loss = mse(prediction, targets)
print(loss)

tensor(8312.6162, grad_fn=<DivBackward0>)


### Compute gradients

In [12]:
loss.backward()
print(weight.grad)
print(bias.grad)

tensor([[-2783.5171, -4655.3501, -2560.2085],
        [-9098.4883, -9828.5020, -6262.1704]])
tensor([ -37.3996, -109.4033])


### Train the model using gradient descent

In [13]:
with torch.no_grad():
    weight -= weight.grad * 1e-5
    bias -= bias.grad * 1e-5
    weight.grad.zero_()
    bias.grad.zero_()

In [14]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(5993.5747, grad_fn=<DivBackward0>)


In [15]:
for i in range(100):
    prediction = model(inputs)
    loss = mse(prediction, targets)
    loss.backward()
    with torch.no_grad():
        weight -= weight.grad * 1e-5
        bias -= bias.grad * 1e-5
        weight.grad.zero_()
        bias.grad.zero_()

In [16]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(459.4715, grad_fn=<DivBackward0>)


In [17]:
prediction

tensor([[ 64.8929,  58.9821],
        [ 84.5650,  69.8854],
        [100.9033, 143.7943],
        [ 66.7419,  30.4835],
        [ 79.1673,  74.9564]], grad_fn=<AddBackward0>)

In [18]:
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

# **Linear regression using PyTorch built-ins**

### Define DataLoader

In [19]:
import numpy as np
import torch

In [20]:
# Targets (apples, oranges)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [21]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [22]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [23]:
from torch.utils.data import TensorDataset

In [24]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [25]:
from torch.utils.data import DataLoader

In [26]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [27]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]])
tensor([[119., 133.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [ 22.,  37.]])


### nn.Linear

In [28]:
import torch.nn as nn

In [29]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1854,  0.1230,  0.3273],
        [-0.5317,  0.2306,  0.3448]], requires_grad=True)
Parameter containing:
tensor([ 0.3782, -0.1723], requires_grad=True)


In [30]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1854,  0.1230,  0.3273],
         [-0.5317,  0.2306,  0.3448]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3782, -0.1723], requires_grad=True)]

In [31]:
preds = model(inputs)
preds

tensor([[  9.1541,  -8.7107],
        [ 15.2720,  -6.1978],
        [ 19.7056,   4.4669],
        [ -1.1381, -31.7334],
        [ 22.2989,   9.4136]], grad_fn=<AddmmBackward0>)

### loss functions

In [32]:
import torch.nn.functional as F

In [33]:
loss_fn = F.mse_loss

In [34]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7436.3096, grad_fn=<MseLossBackward0>)


### Optimizer

In [35]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Train for multiple epochs

In [36]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [37]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 353.8226
Epoch [20/100], Loss: 132.4756
Epoch [30/100], Loss: 113.6313
Epoch [40/100], Loss: 100.4299
Epoch [50/100], Loss: 88.8629
Epoch [60/100], Loss: 78.6709
Epoch [70/100], Loss: 69.6886
Epoch [80/100], Loss: 61.7719
Epoch [90/100], Loss: 54.7938
Epoch [100/100], Loss: 48.6422


In [38]:
prediction = model(inputs)
prediction

tensor([[ 60.0885,  71.4699],
        [ 83.6137,  99.8239],
        [110.7949, 133.0162],
        [ 37.6460,  43.7612],
        [ 94.7764, 113.6960]], grad_fn=<AddmmBackward0>)

In [39]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [40]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[57.8043, 68.6569]], grad_fn=<AddmmBackward0>)